In [1]:
import torch
from test_sparse import *
from utils import *
import json

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rr = []
ss = [0.2, 0.4, 0.6, 0.8]
sd = 0.01
widths, channels, group = regnet_parameters("008")
for i in range(4):
    width, channel = widths[i], channels[i]
    gg = get_factors(width)
    for s in ss:
        for g in gg:
            bs = TVMDynamicBlockEvaluator(channel, width, group, f"log/c{channel}_w{width}_g{group}")
            sl = find_best_sparselen(width//g, s-sd, s+sd)
            bs.setup_sparse(sl, g, True, True, True, True)
            bs.build("sparse")
            res = bs.evaluate("sparse", verbose=True)
            res["channel"] = channel
            res["width"] = width
            res["group"] = group
            res["sparsity"] = s
            res["sparselen"] = sl
            res["granularity"] = g
            rr.append(res)
log_ls_to_json(rr, "results.json")


FileNotFoundError: [Errno 2] No such file or directory: 'log/c64_w56_g16/l40_m4/maskconv.json'

In [3]:
import pandas as pd

# rr = load_json_to_ls("results.json")
df = pd.DataFrame(columns=list(rr[0].keys()))
for r in rr:
    r_ = {k: v for k, v in r.items()}  
    for m in ["conv2", "conv3", "maskconv", "pooling", "gather", "scatter_add"]:
        r_[m] *= 1000 # in ms
    df = df.append(r_, ignore_index=True)
print(df)
df.to_csv("results.csv")

      conv2     conv3    gather scatter_add  maskconv   pooling channel width  \
0  0.028565  0.027888  0.121402    0.117387  0.030312  0.026476    64.0  56.0   
1  0.005817  0.005227  0.011123    0.008683  0.005774  0.006478    64.0  56.0   

  group sparsity sparselen granularity  
0  16.0      0.2     600.0         1.0  
1  16.0      0.2     150.0         2.0  


/tmp/ipykernel_8017/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
/tmp/ipykernel_8017/496098352.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(r_, ignore_index=True)
